In [1]:
import autogen
import openai 
from autogen import AssistantAgent, UserProxyAgent, GroupChatManager, ConversableAgent
import os
from typing_extensions import Annotated


In [2]:
os.environ['OPENAI_API_KEY']="api_key = os.getenv("OPENAI_API_KEY")
print(api_key)


In [18]:




# Change the directories to pick up the files. Ensure you use your own OpenAI API Keys
index_path = r"C:\Users\timveigel\Documents\Masterarbeit\Gabis first assistant\Gabis-MVP Springboot"

configurations_path = r"C:\Users\timveigel\Documents\Masterarbeit\Gabis first assistant\\Gabis-MVP Springboot\\Autogen"




config_list = autogen.config_list_from_json(
    env_or_file="configurations.json",
    file_location=configurations_path,
    filter_dict={
        "model": ["gpt-4-1106-preview"]
    },
)



openai.api_key = api_key


llm_config = {
    "config_list": config_list,
    "seed": 42,
    "timeout": 120
}


# You can also set config_list directly as a list, for example, config_list = [{'model': 'gpt-4', 'api_key': '<your OpenAI API key here>'},]
assistant = AssistantAgent(name="assistant",
                           system_message=f"Fullfill a request. say TERMINATE when you´re done",
                            llm_config={"config_list": config_list},
                            human_input_mode="ALWAYS",
                            is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"))

user_proxy = UserProxyAgent(name="user_proxy",
                            system_message="Terminate when agent says TERMINATE",
                            human_input_mode="NEVER",
                            is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
                            code_execution_config={"work_dir": "stuff", "use_docker": False}) # IMPORTANT: set to True to run code in docker, recommended




@user_proxy.register_for_execution()
@assistant.register_for_llm(description="Writes a hello world program in the given file")
def write_hello_world(code: Annotated[str, "code to be inserted"], file_path: Annotated[str, "path to file, if not specified differently use default"] = ".\\helloworld.py"):
    """
    Inserts the code into the correct file.

    Parameters:
        code (str): The code that should be inserted.
        file_path (str): The path to the file where the code should be inserted.
    """
    with open(file_path, "a") as f:
        f.write(code)

    return "The code has been inserted successfully."


@user_proxy.register_for_execution()
@assistant.register_for_llm(description="Runs a python file")
def run_python_file(file_path: Annotated[str, "path to file"]):
    """
    Runs the given python file.

    Parameters:
        file_path (str): The path to the file that should be run.
    """
    import subprocess
    try:
        result = subprocess.run(["python", file_path], capture_output=True, text=True)
        return result.stdout
    except Exception as e:
        return str(e)


The return type of the function 'write_hello_world' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.
The return type of the function 'run_python_file' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.


In [19]:
the_chat = user_proxy.initiate_chat(assistant, message = 
                         "write a function that prints out the current working directory and execute it. Only terminate when you have executed the function")

user_proxy (to assistant):

write a function that prints out the current working directory and execute it. Only terminate when you have executed the function

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
assistant (to user_proxy):

***** Suggested tool Call (call_7XM3VLR7GaxslMwGQvY55awP): write_hello_world *****
Arguments: 
{"code":"import os\nprint(os.getcwd())","file_path":"./print_cwd.py"}
**********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION write_hello_world...
user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool "call_7XM3VLR7GaxslMwGQvY55awP" *****
The code has been inserted successfully.
**********************************************************************

-----------------------------------------------

In [5]:
assistant.print_usage_summary()
assistant.get_actual_usage()
user_proxy.print_usage_summary()

Agent 'assistant':
----------------------------------------------------------------------------------------------------
No actual cost incurred (all completions are using cache).

Usage summary including cached usage: 
Total cost: 0.00447
* Model 'gpt-4-1106-preview': cost: 0.00447, prompt_tokens: 297, completion_tokens: 50, total_tokens: 347
----------------------------------------------------------------------------------------------------
No cost incurred from agent 'user_proxy'.


In [6]:
print(the_chat.chat_history)


[{'content': 'print out the current working directory.', 'role': 'assistant'}, {'tool_calls': [{'id': 'call_qAIaETLy9jJPGPHrV7YkU9aS', 'function': {'arguments': '{}', 'name': 'print_working_directory'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 'c:\\Users\\timveigel\\Documents\\Masterarbeit\\Gabis first assistant\\Gabis-MVP SpringBoot\\Autogen', 'tool_responses': [{'tool_call_id': 'call_qAIaETLy9jJPGPHrV7YkU9aS', 'role': 'tool', 'content': 'c:\\Users\\timveigel\\Documents\\Masterarbeit\\Gabis first assistant\\Gabis-MVP SpringBoot\\Autogen'}], 'role': 'tool'}, {'content': 'The current working directory is:\n`c:\\Users\\timveigel\\Documents\\Masterarbeit\\Gabis first assistant\\Gabis-MVP SpringBoot\\Autogen`\n\nTERMINATE', 'role': 'user'}]
